In [1]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
import torch.functional as F

import torchvision
from torchvision import transforms

from model import ResNet


In [2]:
# Data loader
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32,padding= 4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

trainloader = DataLoader(train, batch_size=128, shuffle=True, num_workers=2)

test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

testloader = DataLoader(test, batch_size=128,shuffle=False, num_workers=2)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


Files already downloaded and verified
Files already downloaded and verified


In [3]:
#  Model
ResNet50Config = [
    {"in_channels":64,"out_channels":256,"repetition":3,"stride":1},
    {"in_channels":256,"out_channels":512,"repetition":4,"stride":2},
    {"in_channels":512,"out_channels":1024,"repetition":6,"stride":2},
    {"in_channels":1024,"out_channels":2048,"repetition":3,"stride":2},
]
resnet = ResNet(ResNet50Config,len(classes)).to("cuda")


64 64
256 64
256 64
256 128
512 128
512 128
512 128
512 256
1024 256
1024 256
1024 256
1024 256
1024 256
1024 512
2048 512
2048 512


In [4]:
# Train Loop
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor = 0.1, patience=5)

EPOCHS=200
for epoch in range(EPOCHS):
    losses = []
    running_loss = 0
    for index, data in enumerate(trainloader):
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        optimizer.zero_grad()
    
        logits = resnet(inputs)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if index%100 == 0 and index > 0:
            print(f'Loss [{epoch+1}, {index}](epoch, minibatch): ', running_loss / 100)
            running_loss = 0.0

    avg_loss = sum(losses)/len(losses)
    scheduler.step(avg_loss)


Loss [1, 100](epoch, minibatch):  11.597118349075318
Loss [1, 200](epoch, minibatch):  2.864073600769043
Loss [1, 300](epoch, minibatch):  2.280476472377777
Loss [2, 100](epoch, minibatch):  1.9106721556186677
Loss [2, 200](epoch, minibatch):  1.8278814589977264
Loss [2, 300](epoch, minibatch):  1.785675663948059
Loss [3, 100](epoch, minibatch):  1.713610795736313
Loss [3, 200](epoch, minibatch):  1.6760454726219178
Loss [3, 300](epoch, minibatch):  1.64459455370903
Loss [4, 100](epoch, minibatch):  1.632245546579361
Loss [4, 200](epoch, minibatch):  1.5825051426887513
Loss [4, 300](epoch, minibatch):  1.5564600443840027
Loss [5, 100](epoch, minibatch):  1.5502659368515015
Loss [5, 200](epoch, minibatch):  1.4980029487609863
Loss [5, 300](epoch, minibatch):  1.5032159960269929
Loss [6, 100](epoch, minibatch):  1.4675427889823913
Loss [6, 200](epoch, minibatch):  1.460001267194748
Loss [6, 300](epoch, minibatch):  1.4490349721908569
Loss [7, 100](epoch, minibatch):  1.419531899690628
Lo

Loss [52, 100](epoch, minibatch):  0.528058898448944
Loss [52, 200](epoch, minibatch):  0.5560274419188499
Loss [52, 300](epoch, minibatch):  0.5285498562455178
Loss [53, 100](epoch, minibatch):  0.5201211223006248
Loss [53, 200](epoch, minibatch):  0.52774623721838
Loss [53, 300](epoch, minibatch):  0.5943925359845161
Loss [54, 100](epoch, minibatch):  0.5180696535110474
Loss [54, 200](epoch, minibatch):  0.5361387419700623
Loss [54, 300](epoch, minibatch):  0.522774170935154
Loss [55, 100](epoch, minibatch):  0.5113767465949058
Loss [55, 200](epoch, minibatch):  0.5149204495549202
Loss [55, 300](epoch, minibatch):  0.5202765461802482
Loss [56, 100](epoch, minibatch):  0.5110090309381485
Loss [56, 200](epoch, minibatch):  0.5018356102705002
Loss [56, 300](epoch, minibatch):  0.5140234851837158
Loss [57, 100](epoch, minibatch):  0.4890895840525627
Loss [57, 200](epoch, minibatch):  0.5114451947808266
Loss [57, 300](epoch, minibatch):  0.5094378739595413
Loss [58, 100](epoch, minibatch)

Loss [102, 200](epoch, minibatch):  0.37317051261663436
Loss [102, 300](epoch, minibatch):  0.38156033903360365
Loss [103, 100](epoch, minibatch):  0.37098626777529714
Loss [103, 200](epoch, minibatch):  0.3703548565506935
Loss [103, 300](epoch, minibatch):  0.35786140233278274
Loss [104, 100](epoch, minibatch):  0.36499194175004956
Loss [104, 200](epoch, minibatch):  0.3775705170631409
Loss [104, 300](epoch, minibatch):  0.36030038580298424
Loss [105, 100](epoch, minibatch):  0.37266183957457544
Loss [105, 200](epoch, minibatch):  0.37969438821077345
Loss [105, 300](epoch, minibatch):  0.37497862726449965
Loss [106, 100](epoch, minibatch):  0.3527610473334789
Loss [106, 200](epoch, minibatch):  0.35570802450180056
Loss [106, 300](epoch, minibatch):  0.3836536107957363
Loss [107, 100](epoch, minibatch):  0.339065717458725
Loss [107, 200](epoch, minibatch):  0.37012856394052507
Loss [107, 300](epoch, minibatch):  0.3742829164862633
Loss [108, 100](epoch, minibatch):  0.3447384299337864


Loss [151, 300](epoch, minibatch):  0.06104477164335549
Loss [152, 100](epoch, minibatch):  0.05466379531659186
Loss [152, 200](epoch, minibatch):  0.05452787913382053
Loss [152, 300](epoch, minibatch):  0.054464360754936936
Loss [153, 100](epoch, minibatch):  0.052376099405810235
Loss [153, 200](epoch, minibatch):  0.05768597613088786
Loss [153, 300](epoch, minibatch):  0.05997103847563267
Loss [154, 100](epoch, minibatch):  0.05858463745564222
Loss [154, 200](epoch, minibatch):  0.05670063076540828
Loss [154, 300](epoch, minibatch):  0.060091189481317996
Loss [155, 100](epoch, minibatch):  0.06298253939021378
Loss [155, 200](epoch, minibatch):  0.058118418492376804
Loss [155, 300](epoch, minibatch):  0.059695315361022946
Loss [156, 100](epoch, minibatch):  0.0502258543856442
Loss [156, 200](epoch, minibatch):  0.054710500864312055
Loss [156, 300](epoch, minibatch):  0.060102215027436616
Loss [157, 100](epoch, minibatch):  0.04879541135393083
Loss [157, 200](epoch, minibatch):  0.0522

Loss [200, 200](epoch, minibatch):  0.018024649765575305
Loss [200, 300](epoch, minibatch):  0.016658030939288436


In [6]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to('cuda'), labels.to('cuda')
        outputs = resnet(images)
        
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy on 10,000 test images: ', 100*(correct/total), '%')

Accuracy on 10,000 test images:  85.61 %
